<a href="https://colab.research.google.com/github/Alijgh003/StyleMind-GNN/blob/main/edge_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import networkx as nx
import torch
from os import path

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
datasets_path = "/content/drive/MyDrive/StyleMind-GNN"

In [6]:
def build_edge_indices_with_groups(edge_index_tensor_by_groups):
  if(edge_index_tensor_by_groups.shape[0]>0):
    result = torch.cat([edge_index_tensor_by_groups[i] for i in range(edge_index_tensor_by_groups.shape[0])],dim=-1)
    result = result[:,torch.where(result[0]>-1)[0]]
    return result
  else:
    return torch.randn((2, 1))

In [8]:
x = torch.load(path.join(datasets_path, "items_img2vec.pth"))
train_edge_index = torch.load(path.join(datasets_path,"dataset", "polyvore_outfits", "nondisjoint", "train_edge_index.pth"))
x.shape, train_edge_index.shape

(torch.Size([251008, 2048]), torch.Size([53306, 2, 171]))

In [10]:
edge_index= build_edge_indices_with_groups(train_edge_index)
edge_index.shape

torch.Size([2, 686835])

In [12]:
edge_tuples  = list(zip(edge_index[0].tolist(), edge_index[1].tolist()))
edge_tuples

[(204439, 249298),
 (154376, 249298),
 (154376, 204439),
 (154376, 191980),
 (154376, 192270),
 (154376, 6713),
 (154376, 78057),
 (154376, 100757),
 (191980, 249298),
 (191980, 204439),
 (191980, 6713),
 (192270, 249298),
 (192270, 204439),
 (192270, 191980),
 (192270, 6713),
 (6713, 249298),
 (6713, 204439),
 (78057, 249298),
 (78057, 204439),
 (78057, 191980),
 (78057, 192270),
 (78057, 6713),
 (100757, 249298),
 (100757, 204439),
 (100757, 191980),
 (100757, 192270),
 (100757, 6713),
 (100757, 78057),
 (234481, 203733),
 (234481, 182523),
 (234481, 153601),
 (58170, 234481),
 (58170, 203733),
 (58170, 182523),
 (58170, 153601),
 (182523, 203733),
 (175410, 234481),
 (175410, 58170),
 (175410, 203733),
 (175410, 182523),
 (175410, 153601),
 (175410, 33541),
 (153601, 203733),
 (153601, 182523),
 (33541, 234481),
 (33541, 58170),
 (33541, 203733),
 (33541, 182523),
 (33541, 153601),
 (165865, 136159),
 (165865, 16552),
 (165865, 130058),
 (165865, 37489),
 (74889, 165865),
 (74889, 1

In [27]:
graph = nx.Graph()
graph.add_edges_from(edge_tuples)

In [39]:
import random

In [54]:
def get_components_rep(graph, num_of_representatives):
    result = []
    for i in range(num_of_representatives):
      components = nx.connected_components(graph)
      representatives = [list(c)[random.randint(0, len(list(c))-1)] for c in components]
      result.append(representatives)

    return result


In [55]:
def get_augmented_edges(representatives):
  edges_src = list()
  edges_tar = list()
  for rep in representatives:
    for i in range(len(rep)-1):
      edges_src.append(rep[i])
      edges_tar.append(rep[i+1])
  return torch.stack([torch.tensor(edges_src), torch.tensor(edges_tar)])


In [56]:
components_representatives = get_components_rep(graph, 10)
aug_edges = get_augmented_edges(components_representatives)
aug_edges.shape

torch.Size([2, 92250])

In [60]:
aug_edges_tuples = list(zip(aug_edges[0].tolist(),aug_edges[1].tolist()))

In [64]:
torch.save(aug_edges, path.join(datasets_path,"dataset", "polyvore_outfits", "nondisjoint", "train_augmented_edge_index.pth"))

In [61]:
graph.add_edges_from(aug_edges_tuples)

In [63]:
components = nx.connected_components(graph)
len(list(components))

1

In [38]:
sorted(dict(graph.degree()).values(), reverse=True)[:10]

[776, 312, 297, 250, 224, 222, 220, 218, 212, 210]